In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### FIX ME #####
# Change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from mongoCRUD import MongoDBHelper  # Use the MongoDBHelper class from the modified `mongocrud.py`

###########################
# Data Manipulation / Model
###########################

# Connect to database via CRUD Module
mongo_helper = MongoDBHelper(db_name="shelter_database", collection_name="outcomes")

# Check if data is already in MongoDB, if not, import CSV
csv_file_path = 'aac_shelter_outcomes.csv'
if mongo_helper.collection.count_documents({}) == 0:
    mongo_helper.import_csv_to_mongo(csv_file_path)

# Fetch data from MongoDB
df = pd.DataFrame.from_records(mongo_helper.fetch_all_records())
df.drop(columns=['_id'], inplace=True)
columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# FIX ME: Add Grazioso Salvare’s logo
image_filename = 'Grazioso-Salvare-Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# FIX ME: Place the HTML image tag in the line below into the app.layout code according to your design
# Also, include a unique identifier such as your name or date
# html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Center(html.A(
        href='https://snhu.edu',
        target='_blank',  # Open link in a new tab
        children=[
            html.Img(
                src='data:image/png;base64,{}'.format(encoded_image.decode()),
                style={'width': '10%'}
            )
        ]
    )),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Center(html.B(html.P('Colin Aheron'))),
    dcc.Dropdown(['Reset', 
                  'Water Rescue', 
                  'Mountain or Wilderness Rescue',
                  'Disaster or Individual Tracking'], 'Select Rescue Type', id='dropdown'),
    html.Hr(),
    html.Div(),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=columns,
                         data=df.to_dict('records'),
                         page_size=10,
                         sort_action='native',
                         row_selectable="single"),
    html.Br(),
    html.Hr(),
    # Dashboard layout for chart and map side-by-side
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(Output('datatable-id', 'data'),
              [Input('dropdown', 'value')])
def update_dashboard(value):
    # Fetch latest data from MongoDB
    df = pd.DataFrame.from_records(mongo_helper.fetch_all_records())
    df.drop(columns=['_id'], inplace=True)
    
    if value == 'Reset':
        data = df.to_dict('records')
    elif value == 'Water Rescue':
        breeds = ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]
        filtered_df = df[df['breed'].isin(breeds)]
        data = filtered_df.to_dict('records')
    elif value == 'Mountain or Wilderness Rescue':
        breeds = ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]
        filtered_df = df[df['breed'].isin(breeds)]
        data = filtered_df.to_dict('records')
    elif value == 'Disaster or Individual Tracking':
        breeds = ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]
        filtered_df = df[df['breed'].isin(breeds)]
        data = filtered_df.to_dict('records')
    else:
        data = df.to_dict('records')

    return data

# Display the breeds of animals based on quantity represented in the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "data")])
def update_graphs(viewData):
    return [
        dcc.Graph(
            figure=px.pie(df, names='breed', title='Preferred Animals')
        )
    ]

# This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns:
        return []
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# This callback will update the geo-location chart for the selected data entry
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows"),
     Input('datatable-id', 'style_data_conditional')])
def update_map(derived_virtual_data, index, style_data_conditional):
    if not derived_virtual_data:  # Check if data is empty
        raise PreventUpdate

    dff = pd.DataFrame.from_dict(derived_virtual_data)

    # Handle single row selection
    if not index or len(index) == 0:
        row = 0
    else: 
        row = index[0]

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[30.75, -97.48], zoom=10, children=[
               dl.TileLayer(id="base-layer-id"),
               dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                         children=[
                             dl.Tooltip(dff.iloc[row, 4]),
                             dl.Popup([html.H1("Animal Name:"), html.H1(dff.iloc[row, 9])])
                         ])
        ])
    ]

app.run_server(debug=True)


C:\Users\ahero\anaconda3\Lib\site-packages\dash\dash.py:579: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run_server.

In [ ]:
%pip install jupyter_dash
%pip install dash-leaflet
%pip install dash-table
%pip install plotly
%pip install pandas
%pip install matplotlib
%pip install Flask
%pip install Flask-Bcrypt
%pip install Flask-PyMongo

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
